In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# CSV 파일 로드
data = pd.read_csv('/content/drive/MyDrive/data_2024_1.csv', encoding='euc-kr')

# 결측치 처리
data.dropna(subset=["관측시간", "관측지점"], inplace=True)

# 관측시간을 datetime 타입으로 변환
data["관측시간"] = pd.to_datetime(data["관측시간"])

data[["관측지점", "관측지점세부"]] = data["관측지점"].str.split('_', expand=True)

columns_to_fill = ["관측최대풍속"]
columns_to_fill = ["관측최대풍속", "(AVOC)관측최대풍속", "(BVOC)관측최대풍속"]

for column in columns_to_fill:
    data[column] = data.groupby(["관측시간", "관측지점"])[column].transform(
        lambda x: x.fillna(x.mean())
    )

    # 그래도 남아있는 결측치는 관측지점으로 채우기
    data[column] = data.groupby('관측지점')[column].transform(
        lambda x: x.fillna(x.mean())
    )

    # 그래도 남아있는 결측치는 관측시간으로 채우기
    data[column] = data.groupby('관측시간')[column].transform(
        lambda x: x.fillna(x.mean())
    )

    # 여전히 남아있는 결측치는 전체 평균으로 채우기
    overall_mean = data[column].mean()
    data[column].fillna(overall_mean, inplace=True)

In [ ]:
# 결측치 처리 전 결측치 확인
if data.isnull().values.any():
    print("데이터프레임에 결측치가 있습니다.")
else:
    print("데이터프레임에 결측치가 없습니다.")

# 각 열의 결측치 수 확인
missing_values = data.isnull().sum()
print("각 열의 결측치 수:")
print(missing_values)

# 결측치가 있는 열만 출력
# missing_columns = missing_values[missing_values > 0]
# print("결측치가 있는 열과 그 수:")
# print(missing_columns)

In [ ]:
data.sort_values(by="관측시간", inplace=True)

train_ratio = 0.9

train_size = int(len(data) * train_ratio)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

print("훈련 데이터 크기:", len(train_data))
print("테스트 데이터 크기:", len(test_data))

# 아직 관측지점 및 관측지점세부 고려하지 않음
train_data = train_data.drop(columns=['관측지점', '관측지점세부'])
test_data = test_data.drop(columns=['관측지점', '관측지점세부'])

train_data.to_csv("train_data.csv", index=False)
test_data.to_csv("test_data.csv", index=False)